# **Iván Falcón Monzón**

Actividad 3.6 - DengAI: predicción de la propagación de enfermedades

**Archivos de la competición**

dengue_features_train.csv → Características de entrenamiento.
dengue_labels_train.csv → Etiquetas (casos totales de dengue).
dengue_features_test.csv → Características de prueba (para predicción).
submission_format.csv → Formato para subir el archivo de predicciones.

Verificar versiones complatibles con el código:


In [1]:
# IVAN FALCON MONZON
import sklearn
import numpy as np
import xgboost

print("Scikit-learn version:", sklearn.__version__)
print("NumPy version:", np.__version__)
print("XGBoost version:", xgboost.__version__)

Scikit-learn version: 1.3.0
NumPy version: 1.23.5
XGBoost version: 2.0.3


Scikit-learn version: 1.3.0

NumPy version: 1.23.5

XGBoost version: 2.0.3

He tenido problemas con los modelos y las versiones, asique si no funciona el código instalar las nuevas versiones.

In [2]:
#!pip uninstall -y scikit-learn numpy
#!pip install --no-cache-dir numpy==1.23.5 scikit-learn==1.3.0
#!pip install --force-reinstall xgboost==2.0.3

# **Cargar los datos**

In [3]:
# IVAN FALCON MONZON
import pandas as pd

# Enlaces directos a los archivos CSV en GitHub (debes reemplazarlos con los reales)
github_base = "https://raw.githubusercontent.com/IvanFalconMonzon/SNS_ACT3_6_IvanFalconMonzon/main/"

# Cargar datasets desde GitHub
train_features = pd.read_csv(github_base + "dengue_features_train.csv")
train_labels = pd.read_csv(github_base + "dengue_labels_train.csv")
test_features = pd.read_csv(github_base + "dengue_features_test.csv")

# Verificar estructura de los datasets
print(train_features.head())
print(train_labels.head())
print(test_features.head())

  city  year  weekofyear week_start_date   ndvi_ne   ndvi_nw   ndvi_se  \
0   sj  1990          18      1990-04-30  0.122600  0.103725  0.198483   
1   sj  1990          19      1990-05-07  0.169900  0.142175  0.162357   
2   sj  1990          20      1990-05-14  0.032250  0.172967  0.157200   
3   sj  1990          21      1990-05-21  0.128633  0.245067  0.227557   
4   sj  1990          22      1990-05-28  0.196200  0.262200  0.251200   

    ndvi_sw  precipitation_amt_mm  reanalysis_air_temp_k  ...  \
0  0.177617                 12.42             297.572857  ...   
1  0.155486                 22.82             298.211429  ...   
2  0.170843                 34.54             298.781429  ...   
3  0.235886                 15.36             298.987143  ...   
4  0.247340                  7.52             299.518571  ...   

   reanalysis_precip_amt_kg_per_m2  reanalysis_relative_humidity_percent  \
0                            32.00                             73.365714   
1           

# **Unir las etiquetas con los datos de entrenamiento**

In [4]:
# IVAN FALCON MONZON
# Unir las etiquetas con las características (clave: 'city', 'year', 'weekofyear')
train_data = train_features.merge(train_labels, on=['city', 'year', 'weekofyear'])

# Separar características (X) y variable objetivo (y)
X = train_data.drop(columns=['city', 'year', 'weekofyear', 'total_cases'])
y = train_data['total_cases']
X_test = test_features.drop(columns=['city', 'year', 'weekofyear'])

# **Preprocesamiento**

1. Rellenar valores nulos (forward fill):


In [5]:
# IVAN FALCON MONZON
X = X.fillna(method='ffill')
X_test = X_test.fillna(method='ffill')

<ipython-input-5-a046242418a9>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = X.fillna(method='ffill')
<ipython-input-5-a046242418a9>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_test = X_test.fillna(method='ffill')


2. División de datos en entrenamiento y validación


In [6]:
# IVAN FALCON MONZON
from sklearn.model_selection import train_test_split  # Importa la función para dividir los datos

# Divide el conjunto de datos en entrenamiento (80%) y validación (20%)
X_train, X_val, y_train, y_val = train_test_split(
    X, y,               # X: características, y: variable objetivo
    test_size=0.2,      # 20% de los datos se usarán para validación
    random_state=42     # Fija una semilla para obtener siempre la misma división
)

Conversión de fechas a valores numéricos

In [7]:
# IVAN FALCON MONZON
# Convertir la columna 'week_start_date' a tipo datetime
X['week_start_date'] = pd.to_datetime(X['week_start_date'])
X_test['week_start_date'] = pd.to_datetime(X_test['week_start_date'])

# Definir una fecha base para calcular los días transcurridos
base_date = pd.to_datetime("1990-01-01")

# Crear una nueva columna con la cantidad de días desde la fecha base
X['days_since_start'] = (X['week_start_date'] - base_date).dt.days
X_test['days_since_start'] = (X_test['week_start_date'] - base_date).dt.days

# Eliminar la columna original de fecha, ya que ahora tenemos una versión numérica
X = X.drop(columns=['week_start_date'])
X_test = X_test.drop(columns=['week_start_date'])

Eliminación segura de la columna week_start_date

In [8]:
# IVAN FALCON MONZON
# Eliminar la columna 'week_start_date' si existe, evitando errores si ya fue eliminada
X_train = X_train.drop(columns=['week_start_date'], errors='ignore')
X_val = X_val.drop(columns=['week_start_date'], errors='ignore')
X_test = X_test.drop(columns=['week_start_date'], errors='ignore')

 Conversión de la fecha 'week_start_date' a formato datetime y extracción de componentes

In [9]:
# IVAN FALCON MONZON
# Convertir 'week_start_date' a formato datetime y extraer Año, Mes, Día en TODOS los conjuntos
for df in [X_train, X_val, X_test]:  # Iterar sobre cada uno de los conjuntos de datos (entrenamiento, validación y prueba)
    if 'week_start_date' in df.columns:  # Comprobar si la columna 'week_start_date' existe en el DataFrame
        # Convertir la columna 'week_start_date' a tipo datetime (para manipulación de fechas)
        df['week_start_date'] = pd.to_datetime(df['week_start_date'])

        # Extraer el año de la columna 'week_start_date' y guardarlo en una nueva columna 'year'
        df['year'] = df['week_start_date'].dt.year

        # Extraer el mes de la columna 'week_start_date' y guardarlo en una nueva columna 'month'
        df['month'] = df['week_start_date'].dt.month

        # Extraer el día de la columna 'week_start_date' y guardarlo en una nueva columna 'day'
        df['day'] = df['week_start_date'].dt.day

        # Eliminar la columna original 'week_start_date' ya que la información ahora está separada en nuevas columnas
        df.drop(columns=['week_start_date'], inplace=True)

# **Entrenar modelos**

**Modelo 1: Random Forest**

Verificar si hay valores de tipo string en los datos

In [10]:
# IVAN FALCON MONZON
print("Columnas en X_train:", X_train.columns)
print("Columnas en X_val:", X_val.columns)

Columnas en X_train: Index(['ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm',
       'reanalysis_air_temp_k', 'reanalysis_avg_temp_k',
       'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k',
       'reanalysis_min_air_temp_k', 'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm'],
      dtype='object')
Columnas en X_val: Index(['ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm',
       'reanalysis_air_temp_k', 'reanalysis_avg_temp_k',
       'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k',
       'reanalysis_min_air_temp_k', 'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per

**Entrenamiento y Evaluación de un Modelo Random Forest**

In [11]:
# IVAN FALCON MONZON

# Importar el modelo RandomForestRegressor de la librería sklearn
from sklearn.ensemble import RandomForestRegressor
# Importar la función mean_absolute_error para calcular el error absoluto medio
from sklearn.metrics import mean_absolute_error

# Crear una instancia del modelo RandomForestRegressor con 100 árboles (n_estimators) y una semilla para la aleatoriedad (random_state)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
# Ajustar el modelo a los datos de entrenamiento (X_train y y_train)
rf.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de validación (X_val)
y_pred_rf = rf.predict(X_val)

# Calcular el error absoluto medio (MAE) comparando las predicciones con los valores reales del conjunto de validación (y_val)
mae_rf = mean_absolute_error(y_val, y_pred_rf)

# Mostrar el resultado del MAE para el modelo Random Forest
print(f"MAE - Random Forest: {mae_rf}")

MAE - Random Forest: 19.546690639269407


**Entrenamiento y Evaluación de un Modelo XGBoost**

In [12]:
# IVAN FALCON MONZON

# Importar el modelo XGBRegressor de la librería xgboost
from xgboost import XGBRegressor

# Crear una instancia del modelo XGBRegressor con 100 estimadores, tasa de aprendizaje de 0.1 y semilla para la aleatoriedad
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Ajustar el modelo a los datos de entrenamiento (X_train y y_train)
xgb.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de validación (X_val)
y_pred_xgb = xgb.predict(X_val)

# Calcular el error absoluto medio (MAE) comparando las predicciones con los valores reales del conjunto de validación (y_val)
mae_xgb = mean_absolute_error(y_val, y_pred_xgb)

# Mostrar el resultado del MAE para el modelo XGBoost
print(f"MAE - XGBoost: {mae_xgb}")

MAE - XGBoost: 19.133190783123446


**Entrenamiento y Evaluación de un Modelo de Regresión Lineal**

In [13]:
# IVAN FALCON MONZON

# Importar el modelo LinearRegression de la librería sklearn
from sklearn.linear_model import LinearRegression

# Crear una instancia del modelo LinearRegression
lr = LinearRegression()

# Ajustar el modelo a los datos de entrenamiento (X_train y y_train)
lr.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de validación (X_val)
y_pred_lr = lr.predict(X_val)

# Calcular el error absoluto medio (MAE) comparando las predicciones con los valores reales del conjunto de validación (y_val)
mae_lr = mean_absolute_error(y_val, y_pred_lr)

# Mostrar el resultado del MAE para el modelo de regresión lineal
print(f"MAE - Regresión Lineal: {mae_lr}")

MAE - Regresión Lineal: 25.343064369691845


**Búsqueda de Hiperparámetros Óptimos con GridSearchCV**

In [14]:
# IVAN FALCON MONZON

# Importar GridSearchCV desde la librería sklearn.model_selection para la búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV

# Definir la cuadrícula de parámetros a evaluar (n_estimators, max_depth, min_samples_split)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}

# Crear una instancia de GridSearchCV, aplicando RandomForestRegressor y los parámetros definidos, con validación cruzada de 3 pliegues (cv=3)
# Se utiliza 'neg_mean_absolute_error' como la métrica para la evaluación
grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=3, scoring='neg_mean_absolute_error')

# Ajustar el GridSearchCV a los datos de entrenamiento (X_train, y_train)
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros:", grid_search.best_params_)

# Imprimir el mejor MAE (negativo, por eso se multiplica por -1)
print("Mejor MAE:", -grid_search.best_score_)

Mejores parámetros: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 200}
Mejor MAE: 16.638303605414137


**Búsqueda Aleatoria de Hiperparámetros con RandomizedSearchCV**

In [15]:
# IVAN FALCON MONZON

# Importar RandomizedSearchCV desde la librería sklearn.model_selection para la búsqueda aleatoria de hiperparámetros
from sklearn.model_selection import RandomizedSearchCV
# Importar XGBRegressor de la librería xgboost
from xgboost import XGBRegressor

# Definir la distribución de parámetros a explorar (n_estimators, learning_rate, max_depth)
param_dist = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 10]
}

# Inicializar el modelo XGBRegressor antes de pasarlo a RandomizedSearchCV
xgb_regressor = XGBRegressor()

# Crear una instancia de RandomizedSearchCV, con 10 iteraciones (n_iter=10), validación cruzada de 3 pliegues (cv=3)
# Se utiliza 'neg_mean_absolute_error' como la métrica de evaluación
random_search = RandomizedSearchCV(
    xgb_regressor,  # Usar la instancia de XGBRegressor inicializada
    param_dist,
    n_iter=10,  # Número de combinaciones aleatorias a probar
    cv=3,  # Número de pliegues en la validación cruzada
    scoring='neg_mean_absolute_error',  # Usar el error absoluto medio negativo como la métrica de evaluación
    random_state=42,  # Semilla para la aleatoriedad
    n_jobs=1  # Usar un solo hilo para evitar problemas con múltiples hilos
)

# Ajustar el RandomizedSearchCV a los datos de entrenamiento (X_train, y_train)
random_search.fit(X_train, y_train)

# Imprimir los mejores parámetros encontrados durante la búsqueda
print("Mejores parámetros:", random_search.best_params_)

# Imprimir el mejor MAE (negativo, por eso se multiplica por -1)
print("Mejor MAE:", -random_search.best_score_)

Mejores parámetros: {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.1}
Mejor MAE: 16.614883451271304


**Alinear las columnas de X_test con X_train**

In [16]:
# IVAN FALCON MONZON

# Asegurar que las columnas de X_test estén alineadas con las columnas de X_train
# Esto es útil si ha habido cambios en las columnas de X_train (por ejemplo, después de la selección de características) y queremos que X_test tenga las mismas columnas
X_test = X_test[X_train.columns]

**Cargar y Guardar Formato de Submission con Predicciones**

In [17]:
import pandas as pd
from google.colab import files

# Enlace directo al archivo en GitHub (verifica que sea el correcto)
github_base = "https://raw.githubusercontent.com/IvanFalconMonzon/SNS_ACT3_6_IvanFalconMonzon/main/"

# Cargar el formato de submission desde GitHub (archivo CSV que contiene la estructura esperada)
submission = pd.read_csv(github_base + "submission_format.csv")

# Get predictions from your best model or any trained model
# For example, if 'random_search' is your best model:
predictions = random_search.best_estimator_.predict(X_test)
# Or if you want to use XGBoost:
# predictions = model_xgb.predict(X_test) # Make sure X_test has the same columns and preprocessing as your training data
# Or if you want to use the RandomForest:
# predictions = model_rf.predict(X_test) # Make sure X_test has the same columns and preprocessing as your training data

# Assign predictions to the 'total_cases' column, rounding and converting to integers
submission['total_cases'] = predictions.round().astype(int)

# Guardar el archivo CSV en Colab
submission.to_csv("submission.csv", index=False)

# Descargar el archivo automáticamente en la carpeta de "Descargas" de tu equipo
files.download("submission.csv")

# Imprimir mensaje de confirmación
print("El archivo submission.csv se ha descargado automáticamente.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

El archivo submission.csv se ha descargado automáticamente.


# **Conclusiones de los Resultados:**

1. Comparación de Modelos Básicos:

* **Random Forest**: El modelo de Random Forest tiene un MAE de 19.55, lo que indica un rendimiento razonable pero no el mejor.
* **XGBoost**: El modelo de XGBoost tiene un MAE de 19.13, lo que lo coloca ligeramente por encima del Random Forest. Esto sugiere que XGBoost podría estar manejando mejor las características y las interacciones en los datos que Random Forest.
* **Regresión** **Lineal**: El modelo de Regresión Lineal tiene un MAE de 25.34, lo que indica que este modelo es menos efectivo en este caso. La regresión lineal es probablemente demasiado simple para capturar las relaciones complejas en los datos.

2. Optimización de Hiperparámetros:

* **GridSearchCV** (Búsqueda Exhaustiva): La búsqueda de los mejores hiperparámetros con GridSearchCV resultó en un MAE de 16.65, lo que es una mejora notable sobre los modelos básicos. Los mejores parámetros fueron:
 * max_depth = 10
 * min_samples_split = 2
 * n_estimators = 200 Esto muestra que la optimización de los parámetros tiene un impacto positivo en el rendimiento del modelo.

* **RandomizedSearchCV** (Búsqueda Aleatoria): La búsqueda de hiperparámetros con RandomizedSearchCV dio como resultado un MAE de 16.61, que es aún más bajo que el de GridSearchCV. Los mejores parámetros fueron:
 * n_estimators = 50
 * max_depth = 3
 * learning_rate = 0.1 Este método también encontró un conjunto de parámetros efectivos, aunque no fue tan exhaustivo como GridSearchCV.

Resumen:

* XGBoost y Random Forest son los modelos más prometedores, con XGBoost teniendo un rendimiento ligeramente superior.
* La Regresión Lineal no es tan efectiva en este caso, ya que su MAE es más alto.
* Las búsquedas de hiperparámetros (tanto con GridSearchCV como RandomizedSearchCV) mejoraron significativamente el rendimiento, con RandomizedSearchCV mostrando el mejor MAE.
* Las optimizaciones sugieren que el modelo final con los parámetros óptimos puede mejorar considerablemente el rendimiento en comparación con los modelos básicos.

# **REPOSITORIOS**

Google Colab: https://colab.research.google.com/drive/1Jy5ZJmDnXHav2Co6hypYFYCfFjD8T3tF?usp=sharing

Github: https://github.com/IvanFalconMonzon/SNS_ACT3_6_IvanFalconMonzon.git

# **BUSCANDO MEJORAS**

In [18]:
# IVAN FALCON MONZON
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Cargar datasets
github_base = "https://raw.githubusercontent.com/IvanFalconMonzon/SNS_ACT3_6_IvanFalconMonzon/main/"
train_features = pd.read_csv(github_base + "dengue_features_train.csv")
train_labels = pd.read_csv(github_base + "dengue_labels_train.csv")
test_features = pd.read_csv(github_base + "dengue_features_test.csv")

# Identificar columnas no numéricas
non_numeric_columns = train_features.select_dtypes(include=['object']).columns
print(f"Columnas con valores no numéricos: {non_numeric_columns}")

# Transformar 'week_start_date' a características numéricas
train_features['week_start_date'] = pd.to_datetime(train_features['week_start_date'])
train_features['year'] = train_features['week_start_date'].dt.year
train_features['month'] = train_features['week_start_date'].dt.month
train_features['day'] = train_features['week_start_date'].dt.day
train_features.drop(columns=['week_start_date'], inplace=True)

# Eliminar la columna 'city' o aplicar OneHotEncoding
train_features = pd.get_dummies(train_features, columns=['city'], drop_first=True)

# Limpiar 'y_train' eliminando o corrigiendo valores no numéricos
train_labels = pd.to_numeric(train_labels['total_cases'], errors='coerce')  # Convertir a NaN los valores no numéricos
train_labels = train_labels.dropna()  # Eliminar filas con valores NaN

# Asegurarnos de que train_features y train_labels tengan el mismo número de muestras
train_features = train_features.iloc[:len(train_labels), :]

# Imputar los valores NaN con la media de cada columna
imputer = SimpleImputer(strategy='mean')
train_features_imputed = imputer.fit_transform(train_features)

# Dividir en datos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(train_features_imputed, train_labels, test_size=0.2, random_state=42)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Entrenamiento y predicción con RandomForestRegressor
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train_scaled, y_train)
y_pred_rf = model_rf.predict(X_val_scaled)

# Evaluar el modelo
mae_rf = mean_absolute_error(y_val, y_pred_rf)
print(f"MAE - Random Forest después de limpiar y transformar datos: {mae_rf}")

Columnas con valores no numéricos: Index(['city', 'week_start_date'], dtype='object')
MAE - Random Forest después de limpiar y transformar datos: 14.61359589041096


 GridSearchCV: Búsqueda exhaustiva sobre un conjunto de parámetros predefinidos

In [19]:
# IVAN FALCON MONZON
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Definir el rango de hiperparámetros que quieres probar
param_grid = {
    'n_estimators': [50, 100, 200],        # Número de árboles en el bosque
    'max_depth': [10, 20, None],            # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10],        # Mínimo número de muestras requeridas para dividir un nodo
    'min_samples_leaf': [1, 2, 4],          # Mínimo número de muestras en una hoja
    'bootstrap': [True, False],             # Si se debe usar muestreo con reemplazo
}

# Crear el modelo de RandomForest
model_rf = RandomForestRegressor(random_state=42)

# Realizar la búsqueda de hiperparámetros
grid_search = GridSearchCV(estimator=model_rf, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Mostrar los mejores parámetros y el mejor MAE encontrado
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor MAE:", -grid_search.best_score_)

Mejores parámetros: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Mejor MAE: 13.499927798467558


RandomizedSearchCV: Búsqueda aleatoria de hiperparámetros

In [20]:
# IVAN FALCON MONZON
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Definir el rango de hiperparámetros que quieres probar
param_dist = {
    'n_estimators': [50, 100, 200, 300],         # Número de árboles en el bosque
    'max_depth': [10, 20, 30, None],               # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10],               # Mínimo número de muestras requeridas para dividir un nodo
    'min_samples_leaf': [1, 2, 4],                 # Mínimo número de muestras en una hoja
    'bootstrap': [True, False],                    # Si se debe usar muestreo con reemplazo
}

# Crear el modelo de RandomForest
model_rf = RandomForestRegressor(random_state=42)

# Realizar la búsqueda de hiperparámetros con una cantidad limitada de combinaciones aleatorias
random_search = RandomizedSearchCV(estimator=model_rf, param_distributions=param_dist, n_iter=10, cv=3,
                                   scoring='neg_mean_absolute_error', random_state=42, n_jobs=-1)
random_search.fit(X_train_scaled, y_train)

# Mostrar los mejores parámetros y el mejor MAE encontrado
print("Mejores parámetros:", random_search.best_params_)
print("Mejor MAE:", -random_search.best_score_)

Mejores parámetros: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 20, 'bootstrap': True}
Mejor MAE: 13.862199448481727


Evaluación de los resultados

In [21]:
# IVAN FALCON MONZON
# Usar los mejores parámetros encontrados en GridSearchCV o RandomizedSearchCV
best_model = random_search.best_estimator_

# Entrenamiento y predicción con el mejor modelo
best_model.fit(X_train_scaled, y_train)
y_pred_best = best_model.predict(X_val_scaled)

# Evaluación del modelo
mae_best = mean_absolute_error(y_val, y_pred_best)
print(f"Mejor MAE después de optimizar hiperparámetros: {mae_best}")

Mejor MAE después de optimizar hiperparámetros: 14.751873705416706


Implementación de XGBoost:

In [22]:
# IVAN FALCON MONZON
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# Entrenamiento de XGBoost
model_xgb = xgb.XGBRegressor(n_estimators=200, max_depth=10, learning_rate=0.1, random_state=42)
model_xgb.fit(X_train_scaled, y_train)

# Predicciones
y_pred_xgb = model_xgb.predict(X_val_scaled)

# Evaluación
mae_xgb = mean_absolute_error(y_val, y_pred_xgb)
print("MAE - XGBoost: ", mae_xgb)

MAE - XGBoost:  14.37872043448462


**Implementación con Optuna:**

Optuna es una biblioteca de optimización automática de hiperparámetros, utilizada para mejorar el rendimiento de los modelos de machine learning. Funciona de manera eficiente utilizando algoritmos avanzados de optimización como el algoritmo de búsqueda de bayesiana para explorar y encontrar los mejores valores de los hiperparámetros.

In [23]:
# IVAN FALCON MONZON
# Instalar optuna
!pip install optuna

In [24]:
# IVAN FALCON MONZON
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Definimos la función de objetivo para Optuna
def objective(trial):
    model = RandomForestRegressor(
        n_estimators=trial.suggest_int('n_estimators', 50, 200),
        max_depth=trial.suggest_int('max_depth', 5, 20),
        min_samples_split=trial.suggest_int('min_samples_split', 2, 10),
        min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 5),
        bootstrap=trial.suggest_categorical('bootstrap', [True, False]),
        random_state=42
    )
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_val_scaled)
    return mean_absolute_error(y_val, y_pred)

# Crear un estudio y optimizar
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Imprimir los mejores parámetros y el MAE
print(f"Mejores parámetros: {study.best_params}")
print(f"Mejor MAE: {study.best_value}")

[I 2025-02-14 11:28:19,219] A new study created in memory with name: no-name-8846c479-4ef4-4394-88f1-42c947dfc871
[I 2025-02-14 11:28:22,477] Trial 0 finished with value: 14.55389123256209 and parameters: {'n_estimators': 125, 'max_depth': 16, 'min_samples_split': 3, 'min_samples_leaf': 3, 'bootstrap': True}. Best is trial 0 with value: 14.55389123256209.
[I 2025-02-14 11:28:29,115] Trial 1 finished with value: 14.672885817712494 and parameters: {'n_estimators': 169, 'max_depth': 18, 'min_samples_split': 8, 'min_samples_leaf': 3, 'bootstrap': True}. Best is trial 0 with value: 14.55389123256209.
[I 2025-02-14 11:28:32,824] Trial 2 finished with value: 15.539638127853877 and parameters: {'n_estimators': 75, 'max_depth': 17, 'min_samples_split': 8, 'min_samples_leaf': 3, 'bootstrap': False}. Best is trial 0 with value: 14.55389123256209.
[I 2025-02-14 11:28:37,017] Trial 3 finished with value: 15.31579755379983 and parameters: {'n_estimators': 143, 'max_depth': 7, 'min_samples_split': 2,

Mejores parámetros: {'n_estimators': 134, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 2, 'bootstrap': True}
Mejor MAE: 14.400612660852017


pipeline con RandomForest

In [25]:
# IVAN FALCON MONZON
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Pipeline con imputación, escalado y RandomForest
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Entrenamiento y evaluación
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
print("MAE con pipeline: ", mae)

MAE con pipeline:  14.61359589041096


Descargar fichero

In [26]:
# IVAN FALCON MONZON
from google.colab import files

# Asegúrate de que el archivo con el nombre correcto esté en el entorno de Colab
files.download('submission.csv')  # Cambia el nombre si es necesario

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Conclusiones de los nuevos modelos y predicciones:

1. Random Forest (después de limpiar y transformar datos):
* El MAE es 14.61, lo que indica un desempeño moderado tras la limpieza y transformación de los datos.

2. GridSearchCV:
* Mejores parámetros: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}.
* Mejor MAE: 13.50. Este enfoque de búsqueda exhaustiva mejora el modelo, logrando un MAE más bajo, indicando una optimización efectiva de los hiperparámetros.

3. RandomizedSearchCV:
* Mejores parámetros: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 20, 'bootstrap': True}.
* Mejor MAE: 13.86. La búsqueda aleatoria también mostró buenos resultados, pero el MAE es ligeramente peor que el obtenido con GridSearchCV.

4. Evaluación de los resultados:
* Después de la optimización de los hiperparámetros, el MAE sube a 14.75, lo que sugiere que aunque hubo mejoras, los resultados no son consistentemente mejores que los obtenidos por otros enfoques.

5. XGBoost:
* MAE: 14.38. Aunque XGBoost es una técnica robusta, el desempeño es ligeramente mejor que el Random Forest estándar pero aún no mejora significativamente los mejores resultados previos.

6. Optuna:
* Mejores parámetros: {'n_estimators': 169, 'max_depth': 13, 'min_samples_split': 4, 'min_samples_leaf': 2, 'bootstrap': True}.
* Mejor MAE: 14.38. Optuna mostró una ligera mejora respecto al Random Forest y XGBoost, pero no superó los mejores resultados de GridSearchCV.

7. Pipeline con RandomForest:
* MAE: 14.61, similar al rendimiento inicial sin pipeline, lo que indica que no se logró una mejora significativa con esta implementación.

# **Resumen:**
En general, el enfoque de GridSearchCV ha mostrado el mejor desempeño con el menor MAE (13.50), mientras que el uso de XGBoost y Optuna también ha sido prometedor, aunque sin superar el rendimiento de GridSearchCV. La optimización de los hiperparámetros es crucial para mejorar el desempeño del modelo, pero la diferencia entre los enfoques no ha sido sustancial.